In [21]:
import pandas as pd

df = pd.read_csv('flights.csv')
df

,Airline,Flight Number,Departure City,Arrival City,Departure Date,Return Date,Price,Stops,Duration,Departure Time,Arrival Time
0,American,AA 1299,Austin-Bergstrom International Airport,Dallas Fort Worth International Airport,2024-11-23 19:30,NaN,295,0,71,2024-11-23 19:30,2024-11-23 20:41
1,American,AA 1228,Austin-Bergstrom International Airport,Dallas Fort Worth International Airport,2024-11-23 20:35,NaN,295,0,73,2024-11-23 20:35,2024-11-23 21:48
2,United,UA 6321,Austin-Bergstrom International Airport,George Bush Intercontinental Airport,2024-11-23 19:34,NaN,304,1,66,2024-11-23 19:34,2024-11-23 20:40
3,United,UA 6116,George Bush Intercontinental Airport,Dallas Fort Worth International Airport,2024-11-23 21:37,NaN,304,1,83,2024-11-23 21:37,2024-11-23 23:00
4,American,AA 2434,Austin-Bergstrom International Airport,Dallas Fort Worth International Airport,2024-11-23 15:20,NaN,369,0,70,2024-11-23 15:20,2024-11-23 16:30
...,...,...,...,...,...,...,...,...,...,...,...
819,Delta,DL 5765,John F. Kennedy International Airport,Boston Logan International Airport,2025-02-27 06:00,NaN,267,0,90,2025-02-27 06:00,2025-02-27 07:30
820,Delta,DL 5733,John F. Kennedy International Airport,Boston Logan International Airport,2025-02-27 09:50,NaN,302,0,80,2025-02-27 09:50,2025-02-27 11:10
821,Delta,DL 5813,John F. Kennedy International Airport,Boston Logan International Airport,2025-02-27 14:30,NaN,302,0,84,2025-02-27 14:30,2025-02-27 15:54
822,Delta,DL 2639,John F. Kennedy International Airport,Boston Logan International Airport,2025-02-27 21:00,NaN,302,0,87,2025-02-27 21:00,2025-02-27 22:27


In [20]:
df.Airline.value_counts()
df['Departure City'].value_counts()

Departure City
San Francisco International Airport                             129
Dallas Fort Worth International Airport                          96
Chicago O'Hare International Airport                             85
Los Angeles International Airport                                84
John F. Kennedy International Airport                            81
Miami International Airport                                      70
Denver International Airport                                     64
Seattle Tacoma International Airport                             61
Hartsfield-Jackson Atlanta International Airport                 52
Boston Logan International Airport                               51
Austin-Bergstrom International Airport                           12
George Bush Intercontinental Airport                             11
Charlotte Douglas International Airport                           6
Phoenix Sky Harbor International Airport                          5
Salt Lake City International Airp